# Ch 18: Subtyping

We will practice subtyping, operator overloading, and other topics from the previous chapter by developing parts of a program for playing card games.

### Cards

Perhaps the most important objects we need to deal with in this program are cards.  So we need to first decide on a new type that can effectively represent these cards.

There are fifty-two cards in a deck, each of which belongs to one of four suits and one of thirteen ranks. The suits are Spades ( ♠ ), Hearts ( ♥ ), Diamonds ( ♦ ), and Clubs ( ♣ ). The ranks are Ace (A), 2, 3, 4, 5, 6, 7, 8, 9, 10, Jack (J), Queen (Q), and King (K). Depending on the game that you are playing, an Ace may be higher than King or lower than 2.

If we want to define a new object to represent a playing card, it is obvious what the attributes should be: rank and suit. It is not as obvious what type the attributes should be. One possibility is to use strings containing words like "Spade" for suits and "Queen" for ranks. One problem with this implementation is that it would not be easy to compare cards to see which had a higher rank or suit.

An alternative is to use integers to encode the ranks and suits. In this context, “encode” means that we are going to define a mapping between numbers and suits, or between numbers and ranks. This kind of encoding is not meant to be a secret (that would be “encryption”).

For example, this table shows the suits and the corresponding integer codes:

♠ $\rightarrow$ 4

♥ $\rightarrow$ 3

♦ $\rightarrow$ 2

♣ $\rightarrow$ 1

This code makes it easy to compare cards; because higher suits map to higher numbers, we can compare suits by comparing their codes.

In [1]:
struct Card
    
    suit :: Int64
    rank :: Int64
    
    function Card(suit::Int64, rank::Int64)
        @assert(1 ≤ suit ≤ 4, "suit is not between 1 and 4")
        @assert(1 ≤ rank ≤ 13, "rank is not between 1 and 13")
        new(suit, rank)
    end
    
end

### Global Variables

In order to print Card objects in a way that people can easily read, we need a mapping from the integer codes to the corresponding ranks and suits. A natural way to do that is with arrays of strings:

In [2]:
const suit_names = ["♣", "♦", "♥", "♠"]
const rank_names = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]

13-element Array{String,1}:
 "A"
 "2"
 "3"
 "4"
 "5"
 "6"
 "7"
 "8"
 "9"
 "10"
 "J"
 "Q"
 "K"

The variables suit_names and rank_names are global variables. The `const` declaration means that the variable can only
be assigned once. This solves the performance problem of global variables.

Now we can implement an appropriate show method:

In [3]:
function Base.show(io::IO, card::Card)
    print(io, rank_names[card.rank], suit_names[card.suit])
end

In [4]:
Card(3, 11)

J♥

### Comparing Cards

For built-in types, there are relational operators ( < , > , == , etc.) that compare values and determine when one is greater than, less than, or equal to another. For programmer-defined types, we can override the behavior of the built-in operators by providing a method named: `<`.

The correct ordering for cards is not obvious. For example, which is better, the 3 of Clubs or the 2 of Diamonds? One has a higher rank, but the other has a higher suit. In order to compare cards, you have to decide whether rank or suit is more
important. 

The answer might depend on what game you are playing, but to keep things simple, we’ll make the arbitrary choice that the suit is more important, so all of the Spades outrank all of the Diamonds, and so on. With that decided, we can write `<`:

In [5]:
import Base.<
function <(c1::Card, c2::Card)
    (c1.suit, c1.rank) < (c2.suit, c2.rank)
end

< (generic function with 76 methods)

### Unit Testing

Unit testing allows you to verify the correctness of your code by comparing the results of your code to what you expect. This can be useful to be sure that your code still is correct after modifications, and it is also a way to predefine the correct behaviour of your code during development. 

Simple unit testing can be performed with the @test macros:

In [6]:
using Test
@test Card(1, 4) < Card(2, 4)

Test Passed

In [7]:
@test Card(1, 3) < Card(1, 4)

Test Passed

### Abstract Types and Subtyping

Before we define a type "Card," we also need to think about how to handle two other critical objects: a deck and a hand.  A hand is a set of the cards held by one player. 

A hand is similar to a deck: both are made up of a collection of cards, and both require operations like adding and removing cards.

A hand is also different from a deck; there are operations we want for hands that don’t make sense for a deck. For
example, in poker we might compare two hands to see which one wins. In bridge, we might compute a score for a hand in
order to make a bid.

So we need a way to group these two related concrete types. In Julia this is done by defining an abstract type that serves as a parent for both `Deck` and `Hand`.  As you may recall, this is subtyping.

Let’s call this abstract type `CardSet`:


In [8]:
abstract type CardSet end

There are several kinds of relationship between types:

* Objects of a concrete type might contain references to objects of another type. For example, each `Rectangle` contains a reference to a `Point`, and each `Deck` contains references to an array of `Card`s. This kind of relationship is called HAS-A, as in, “a Rectangle has a Point”.

* A concrete type can have an abstract type as a supertype. This relationship is called IS-A, as in, “a Hand is a kind of a CardSet.”

* One type might depend on another in the sense that objects of one type take objects of the second type as parameters, or use objects of the second type as part of a computation. This kind of relationship is called a dependency.

Let's now define some methods that can be used for all subtypes of `CardSet`:

In [9]:
function Base.show(io::IO, cs::CardSet)
    for card in cs.cards   # every subtypes of CardSet should have cards
        print(io, card, " ")   # we already defined how to show a card
    end
    println()
end


function Base.pop!(cs::CardSet)
    pop!(cs.cards)
end


function Base.push!(cs::CardSet, card::Card)
    push!(cs.cards, card)
    nothing
end


Methods like `pop!` and `push!` for CardSet that use another method without doing much work is sometimes called a **veneer**. The metaphor comes from woodworking, where a veneer is a thin layer of good quality wood glued to the surface of a cheaper piece of wood to improve the appearance.

In this case these methods merely express array operations in terms appropriate for cardsets.  It improves the appearance, or interface, of the implementation.

In [10]:
struct Deck <: CardSet
    cards :: Array{Card, 1}
end

function Deck()  # an outer constructor with no argument
    cards =[]
    for suit in 1:4
        for rank in 1:13
            push!(cards, Card(suit, rank))  # generate a card instance
        end
    end
    deck = Deck(cards)
end

Deck

In [11]:
deck1 = Deck();

deck1 isa CardSet

true

In [12]:
deck1

A♣ 2♣ 3♣ 4♣ 5♣ 6♣ 7♣ 8♣ 9♣ 10♣ J♣ Q♣ K♣ A♦ 2♦ 3♦ 4♦ 5♦ 6♦ 7♦ 8♦ 9♦ 10♦ J♦ Q♦ K♦ A♥ 2♥ 3♥ 4♥ 5♥ 6♥ 7♥ 8♥ 9♥ 10♥ J♥ Q♥ K♥ A♠ 2♠ 3♠ 4♠ 5♠ 6♠ 7♠ 8♠ 9♠ 10♠ J♠ Q♠ K♠ 

In [13]:
struct Hand <: CardSet
    cards :: Array{Card, 1}
    label :: String
end

function Hand(label::String="")     # an outer constructor.  
    Hand(Card[], label)     # Note that the interface (just one argument) is different from that of the default constructors
end

Hand

In [14]:
Array{Card,1}()

0-element Array{Card,1}

In [15]:
Card[]

0-element Array{Card,1}

Let's now define a method called `move!` to deal cards:

In [16]:
function move!(cs1::CardSet, cs2::CardSet, n::Int)
    @assert 1 ≤ n ≤ length(cs1.cards)
    
    for i in 1:n
        card = pop!(cs1)
        push!(cs2, card)
    end
    nothing
end

move! (generic function with 1 method)

In [17]:
deck_m = Deck()
hand_m = Hand("testing move!")

move!(deck_m, hand_m,2)
hand_m

K♠ Q♠ 

And finally, lets' define a couple of methods specific to Deck:

In [18]:
using Random

function shuffle!(deck::Deck)
    Random.shuffle!(deck.cards)
    deck
end

shuffle! (generic function with 1 method)

In [19]:
shuffle!(deck1)

K♠ 4♠ 7♥ A♠ 3♠ 5♣ J♣ 4♦ 7♦ A♣ J♠ 9♦ Q♥ 2♣ K♦ 3♦ 5♦ A♦ 6♥ 4♥ 6♦ Q♦ 10♦ Q♠ 9♥ 3♣ 2♥ K♣ J♥ 3♥ 9♠ 2♠ 10♥ 2♦ 7♠ 7♣ 8♣ 6♠ 6♣ 8♥ K♥ 4♣ A♥ 8♠ 10♠ 5♠ J♦ 9♣ Q♣ 8♦ 5♥ 10♣ 

### Exercise 18-2

Write a function named `sort!` that uses the function `sort!` to sort the cards in a Deck. `sort!` uses the `isless` method to determine the order.

In [20]:
function Base.sort!(deck::Deck)
    sort!(deck.cards)
    deck
end

In [21]:
import Base.isless
isless(c1::Card, c2::Card) = c1 < c2

isless (generic function with 42 methods)

In [22]:
sort!(deck1)

A♣ 2♣ 3♣ 4♣ 5♣ 6♣ 7♣ 8♣ 9♣ 10♣ J♣ Q♣ K♣ A♦ 2♦ 3♦ 4♦ 5♦ 6♦ 7♦ 8♦ 9♦ 10♦ J♦ Q♦ K♦ A♥ 2♥ 3♥ 4♥ 5♥ 6♥ 7♥ 8♥ 9♥ 10♥ J♥ Q♥ K♥ A♠ 2♠ 3♠ 4♠ 5♠ 6♠ 7♠ 8♠ 9♠ 10♠ J♠ Q♠ K♠ 

### Debugging

Subtyping can make debugging difficult because when you call a function with an object as argument, it might be hard to figure out which method will be invoked.

Any time you are unsure about the flow of execution through your program, the simplest solution is to add print statements at the beginning of the relevant methods. If shuffle! prints a message that says something like Running shuffle! Deck , then as the program runs it traces the flow of execution. 

As better alternative, you can also use the @which macro:

In [27]:
deckD = Deck()
hand = Hand("try which")
move!(deckD, hand,10)
hand

K♠ Q♠ J♠ 10♠ 9♠ 8♠ 7♠ 6♠ 5♠ 4♠ 

In [28]:
function Base.sort!(hand::Hand)
    sort!(hand.cards)
    hand
end

In [29]:
@which sort!(hand)

sort!(hand::Hand) in Main at In[28]:2

**Here’s a design suggestion:**

When you override a method, the interface of the new method should be the same as the old. It should take the same parameters, return the same type, and obey the same preconditions and postconditions. If you follow this rule, you will find that any function designed to work with an instance of a supertype, like a `CardSet`, will also work with instances of its subtypes `Deck` and `Hand`. 

If you violate this rule, which is called the “Liskov substitution principle”, your code will collapse like (sorry) a house of
cards.

### Exercise 18-5

Write a method called `deal!` that takes three parameters, a Deck, the number of hands and the number of cards per hand. It should create the appropriate number of Hand objects, deal the appropriate number of cards per hand, and return an array of Hands.

In [48]:
function deal!(deck::Deck, n::Int64, m::Int64)
    
    @assert n*m < length(deck.cards)
    
    shuffle!(deck)
    
    hands_array = Hand[]
    
    for ind_hand = 1:n
        
        push!(hands_array, Hand("Hand-$(ind_hand)"))
        move!(deck, hands_array[ind_hand], m)
        
    end
    
    return hands_array
    
end
        

deal! (generic function with 1 method)

In [47]:
deck_deal1 = Deck()
deal!(deck_deal1, 4, 7)

4-element Array{Hand,1}:
 2♣ 4♠ 9♦ 8♠ 6♥ A♣ 5♥  
 K♦ 4♣ 3♠ Q♣ 3♥ 10♣ 8♥ 
 9♥ 7♦ J♠ 9♣ 8♦ Q♦ A♠  
 5♠ 7♠ K♠ 10♥ 3♦ K♥ 6♠ 

### Exercise 18-6

The following are the possible hands in poker, in increasing order of value and decreasing order of probability:

- pair: two cards with the same rank

- two pair: two pairs of cards with the same rank

- three of a kind: three cards with the same rank

- straight: five cards with ranks in sequence (aces can be high or low, so Ace-2-3-4-5 is a straight and so is 10-Jack-Queen-King-Ace, but Queen-King-Ace-2-3 is not.)

- flush: five cards with the same suit

- full house: three cards with one rank, two cards with another

- four of a kind: four cards with the same rank

- straight flush: five cards in sequence (as defined above) and with the same suit

The goal of this exercise is to estimate the probability of drawing these various hands.

1. Add methods named `haspair`, `hastwopair`, etc. that return true or false according to whether or not the hand meets the relevant criteria. Your code should work correctly for “hands” that contain any number of cards (although 5 and 7 are the most common sizes).

2. Write a method named `classify` that figures out the highest-value classification for a hand and sets the label field accordingly. For example, a 7-card hand might contain a flush and a pair; it should be labeled “flush”.

3. When you are convinced that your classification methods are working, the next step is to estimate the probabilities of the various hands. Write a function that shuffles a deck of cards, divides it into hands, classifies the hands, and counts the number of times various classifications appear.

4. Print a table of the classifications and their probabilities. Run your program with larger and larger numbers of hands until the output values converge to a reasonable degree of accuracy. Compare your results to the values at
https://en.wikipedia.org/wiki/Hand_rankings.


TIP: First, make up test hands for each classification.

Second, write a function `process_hands` to build a histogram (for the ranks of cards) with a hand as an argument.  Use this to classify pair, two pair, three of a kind, four of a kind and  full house.  Test these methods using the test hands.

Third, write a function `hasflush` that first buils a histogram (for the suits of cards).

Fourth, write a function `hasstraight` using ∈ to check all correct ranks are in an array of the ranks in the hand.  We cannot use typical card program algorithms/tricks because our hands can have any number of cards (not just 5 or 6).



In [270]:
pair_hand = Hand([Card(1,2), Card(2,2), Card(1,1), Card(3,3), Card(4,4), Card(2,8), Card(3,7)],"pair")
twopair_hand = Hand([Card(1,2), Card(2,2), Card(3,1), Card(4,1), Card(3,4), Card(1,5), Card(4,7)],"twopair")
threeofakind_hand = Hand([Card(1,2), Card(2,2), Card(3,2), Card(4,1), Card(3,4), Card(1,5), Card(4,7)],"threeofakind")
fourofakind_hand = Hand([Card(1,2), Card(2,2), Card(3,2), Card(4,2), Card(3,4), Card(1,5), Card(4,7)],"fourofakind")
fullhouse_hand = Hand([Card(1,2), Card(2,2), Card(3,2), Card(4,3), Card(3,3), Card(1,5), Card(4,7)],"fullhouse")

flush_hand = Hand([Card(1,2), Card(1,3), Card(1,4), Card(1,5), Card(1,10), Card(2,11), Card(4,7)],"flush")

#Ace-2-3-4-5: straight
#10-Jack-Queen-King-Ace: stright
#Queen-King-Ace-2-3: NO straight

straight_hand1 = Hand([Card(1,4), Card(2,5), Card(3,6), Card(4,7), Card(1,8), Card(1,1), Card(4,10)],"straight")
straight_hand2 = Hand([Card(1,1), Card(2,2), Card(3,3), Card(4,4), Card(1,5), Card(1,10), Card(4,7)],"straight")
straight_hand3 = Hand([Card(1,10), Card(2,11), Card(3,12), Card(4,13), Card(2,1), Card(3,9), Card(4,7)],"straight")
straight_hand4 = Hand([Card(1,12), Card(2,13), Card(3,1), Card(4,2), Card(2,3), Card(3,10), Card(4,7)],"nostraight")
straight_hand5 = Hand([Card(1,4), Card(2,5), Card(3,6), Card(4,8), Card(2,9), Card(3,10), Card(4,12)],"nostraight")

straightflush_hand1 = Hand([Card(1,4), Card(1,5), Card(1,6), Card(1,7), Card(1,8), Card(2,1), Card(4,10)],"straightflush")
straightflush_hand2 = Hand([Card(1,2), Card(1,3), Card(1,5), Card(2,7), Card(1,4), Card(1,6), Card(4,10)],"straightflush")
straightflush_hand3 = Hand([Card(3,10), Card(3,11), Card(3,12), Card(3,13), Card(1,5), Card(3,1), Card(4,7)],"straightflush")
straightflush_hand4 = Hand([Card(1,12), Card(2,13), Card(3,1), Card(3,2), Card(3,3), Card(3,4), Card(4,7)],"straightflush")
straightflush_hand5 = Hand([Card(2,4), Card(1,5), Card(1,6), Card(1,7), Card(1,8), Card(3,10), Card(4,12)],"straightbutnoflush")


4♦ 5♣ 6♣ 7♣ 8♣ 10♥ Q♠ 

In [271]:
hands = [pair_hand, twopair_hand, threeofakind_hand, fourofakind_hand, fullhouse_hand, flush_hand,
straight_hand1,straight_hand2,straight_hand3,straight_hand4,straight_hand5,
straightflush_hand1,straightflush_hand2,straightflush_hand3,straightflush_hand4,straightflush_hand5]

16-element Array{Hand,1}:
 2♣ 2♦ A♣ 3♥ 4♠ 8♦ 7♥  
 2♣ 2♦ A♥ A♠ 4♥ 5♣ 7♠  
 2♣ 2♦ 2♥ A♠ 4♥ 5♣ 7♠  
 2♣ 2♦ 2♥ 2♠ 4♥ 5♣ 7♠  
 2♣ 2♦ 2♥ 3♠ 3♥ 5♣ 7♠  
 2♣ 3♣ 4♣ 5♣ 10♣ J♦ 7♠ 
 4♣ 5♦ 6♥ 7♠ 8♣ A♣ 10♠ 
 A♣ 2♦ 3♥ 4♠ 5♣ 10♣ 7♠ 
 10♣ J♦ Q♥ K♠ A♦ 9♥ 7♠ 
 Q♣ K♦ A♥ 2♠ 3♦ 10♥ 7♠ 
 4♣ 5♦ 6♥ 8♠ 9♦ 10♥ Q♠ 
 4♣ 5♣ 6♣ 7♣ 8♣ A♦ 10♠ 
 2♣ 3♣ 5♣ 7♦ 4♣ 6♣ 10♠ 
 10♥ J♥ Q♥ K♥ 5♣ A♥ 7♠ 
 Q♣ K♦ A♥ 2♥ 3♥ 4♥ 7♠  
 4♦ 5♣ 6♣ 7♣ 8♣ 10♥ Q♠ 

In [239]:
function process_hands(hand::Hand)
    
    dict = Dict()
    for card in hand.cards
        dict[card.rank] = get!(dict, card.rank, 0) + 1
    end
    
    pairs = findall(isequal(2),dict)
    trio = findall(isequal(3),dict)
    quad = findall(isequal(4),dict)
    
    return pairs,trio,quad
    
end


function haspair(hand::Hand)
    
    pairs,trio,quad = process_hands(hand)
    
    length(pairs) == 1 && length(trio) != 1
        
end 

function hastwopair(hand::Hand)
    
    pairs,trio,quad = process_hands(hand)
    
    length(pairs) == 2
        
end 

function hasthreeofakind(hand::Hand)
    
    pairs,trio,quad = process_hands(hand)
    
    length(trio) == 1 && length(pairs) != 1
        
end 

function hasfourofakind(hand::Hand)
    
    pairs,trio,quad = process_hands(hand)
    
    length(quad) == 1
        
end 

function hasfullhouse(hand::Hand)
    
    pairs,trio,quad = process_hands(hand)
    
    length(trio) == 1 && length(pairs) == 1
        
end 

function hasflush(hand::Hand)
    
    dict = Dict()
    for card in hand.cards
        dict[card.suit] = get!(dict, card.suit, 0) + 1
    end
    
     length(findall(isequal(5),dict)) >= 1
    
end

hasflush (generic function with 1 method)

In [267]:
function hasstraight(hand::Hand)
    
    s = []
    for card in hand.cards
        push!(s,card.rank)
    end
    
    if 1 ∈ s && 10 ∈ s && 11 ∈ s && 12 ∈ s && 13 ∈ s
        return true
    end
        
    
    for i = 1:length(s)
                
        if s[i] + 1 ∈ s && s[i] + 2 ∈ s && s[i] + 3 ∈ s && s[i] + 4 ∈ s
            return true
        end
        
    end
            
    
    return false
    
end

hasstraight (generic function with 1 method)

In [254]:
function hasstraightflush(hand::Hand)
    
    dict = Dict()
    for card in hand.cards
        dict[card.suit] = get!(dict, card.suit, 0) + 1
    end
    
    flushsuits = findall(isequal(5),dict)

    for flushsuit in flushsuits
        flush_cards = filter(p->p.suit == flushsuit, hand.cards)
        if hasstraight(Hand(flush_cards,"test"))
            return true
        end
    end
    
    return false
    
end
    


hasstraightflush (generic function with 1 method)

In [272]:
for hand in hands
    print("$(haspair(hand))  ")
end
    

true  false  false  false  false  false  false  false  false  false  false  false  false  false  false  false  

In [273]:
for hand in hands
    print("$(hastwopair(hand))  ")
end

false  true  false  false  false  false  false  false  false  false  false  false  false  false  false  false  

In [274]:
for hand in hands
    print("$(hasthreeofakind(hand))  ")
end

false  false  true  false  false  false  false  false  false  false  false  false  false  false  false  false  

In [275]:
for hand in hands
    print("$(hasfourofakind(hand))  ")
end

false  false  false  true  false  false  false  false  false  false  false  false  false  false  false  false  

In [276]:
for hand in hands
    print("$(hasfullhouse(hand))  ")
end

false  false  false  false  true  false  false  false  false  false  false  false  false  false  false  false  

In [277]:
for hand in hands
    print("$(hasflush(hand))  ")
end

false  false  false  false  false  true  false  false  false  false  false  true  true  true  false  false  

In [278]:
for hand in hands
    print("$(hasstraight(hand))  ")
end

false  false  false  false  false  false  true  true  true  false  false  true  true  true  false  true  

In [279]:
for hand in hands
    print("$(hasstraightflush(hand))  ")
end

false  false  false  false  false  false  false  false  false  false  false  true  true  true  false  false  

Write a function that shuffles a deck of cards, divides it into hands, classifies the hands, and counts the number of times various classifications appear.

In [280]:
count_array = Int.(zeros(8))  # pair, twopair, threeofakind, fourofakind, fullhouse, stright, flush, straightflush

numberhands = 10
numbertrials = 100000

for trials = 1:numbertrials
    
    hands = deal!(Deck(), numberhands, 5)
    for hand in hands
        if haspair(hand)
            count_array[1] += 1
        end
        if hastwopair(hand)
            count_array[2] += 1
        end
        if hasthreeofakind(hand)
            count_array[3] += 1
        end
        if hasfourofakind(hand)
            count_array[4] += 1
        end
        if hasfullhouse(hand)
            count_array[5] += 1
        end
        if hasstraight(hand) && ! hasstraightflush(hand)
            count_array[6] += 1
        end
        if hasflush(hand) && ! hasstraightflush(hand)
            count_array[7] += 1
        end
        if hasstraightflush(hand)
            count_array[8] += 1
        end
    end
        
end

probability = count_array ./(numberhands*numbertrials)

8-element Array{Float64,1}:
 0.422799
 0.047598
 0.021319
 0.000223
 0.001468
 0.00396 
 0.001985
 1.3e-5  

For 5 cards

Hand	Probability	Number of Hands

Single Pair	0.422569	1098240

Two Pair	0.047539	123552

Triple	0.0211285	54912

Four of a Kind	0.000240096	624

Full House	0.00144058	3744

Straight(excluding Straight Flush and Royal Flush)	0.00392465	10200

Flush (but not a Straight)	0.0019654	5108

Straight Flush (but not Royal)	0.0000138517	36

<Royal Flush	0.00000153908	4>: not considered

### Data Encapsulation

In our example on Markov analys, we used two variables— suffixes and prefix —that are read and written from several functions.

If these variables are global, we can only run one analysis at a time. If we read two texts, their prefixes and suffixes would be added to the same data structures (which makes for some interesting generated text). 

To run multiple analyses, and keep them separate, we can encapsulate the state of each analysis in an object. Here’s what that looks like:

In [43]:
struct Markov
    order :: Int64
    suffixes :: Dict{Tuple{String,Vararg{String}}, Array{String, 1}}
    prefix :: Array{String, 1}
end

function Markov(order::Int64=2)
    new(order, Dict{Tuple{String,Vararg{String}}, Array{String, 1}}(), Array{String, 1}())
end

Markov

Next, we transform the functions into methods. For example, here’s processword :

Transforming a program like this—changing the design without changing the behavior—is another example of refactoring
(see Refactoring). 

In [ ]:
function processword(markov::Markov, word::String)
    
    if length(markov.prefix) < markov.order
        push!(markov.prefix, word)
        return
    end
    
    get!(markov.suffixes, (markov.prefix...,), Array{String, 1}())
    push!(markov.suffixes[(markov.prefix...,)], word)
    popfirst!(markov.prefix)
    push!(markov.prefix, word)
    
end

This example suggests a development plan for designing types:

* Start by writing functions that read and write global variables (when necessary).

* Once you get the program working, look for associations between global variables and the functions that use them.

* Encapsulate related variables as fields of a struct.

* Transform the associated functions into methods with as argument objects of the new type.
